In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import time
import amt.method as md
import amt.data_loader as dl
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the data

In [2]:
output_folder = '../../results/GWAS'
output_file = output_folder + '/small_GWAS_chr4.pickle'
with open('../../parkinsons/parkinsons.pickle', 'rb') as f:
    X = pickle.load(f)
    y = pickle.load(f)
    miss_prop = pickle.load(f)
y = y-1
file_map = '../../parkinsons/parkinsons.map'
df_map = pd.read_csv(file_map, delimiter='\t', 
                     names=['chromosome', 'snp', 'start', 'end'])
n_sample, n_snp = X.shape
ind_small = np.array(df_map['chromosome']==4, dtype=bool)
ind_snp = np.array(miss_prop[ind_small]<0.05, dtype=bool)
n_hypothesis = np.sum(ind_snp)

/home/martin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Compute the expected observations
Exp = np.zeros([8, n_snp], dtype=float)
for iy in range(2):
    for ix in range(4):
        Exp[iy*4+ix,:] = np.mean(y==iy) * np.mean(X==ix,axis=0)
Exp = Exp*n_sample
r_Exp = 1/Exp.clip(min=1e-6)*(Exp>0)
chi2_obs = md.compute_chi2(y, X, Exp, r_Exp)
data_gwas = {'X':X, 'y':y, 'Exp':Exp, 'r_Exp':r_Exp, 'chi2_obs':chi2_obs}

# Compute fMC p-values

In [4]:
n_fMC = int(2.5e5)
# n_fMC = int(5e5)
data_gwas_small = {'X':X[:,ind_small], 'y':y, 'Exp':Exp[:,ind_small],
                   'r_Exp':r_Exp[:,ind_small], 'chi2_obs':chi2_obs[ind_small]}

In [5]:
np.random.seed(0)
start_time = time.time()
B = md.permute_chi2_batch_ncore(data_gwas_small['y'],
                                data_gwas_small['X'],
                                data_gwas_small['Exp'],
                                data_gwas_small['r_Exp'],
                                data_gwas_small['chi2_obs'], n_fMC,
                                verbose=True,
                                n_core=32)
p_fmc = (np.sum(B, axis=0)+1)/(n_fMC+1)
print('# Time=%0.1fs'%(time.time()-start_time))
time_fMC = time.time()-start_time
res_fMC = {'time':time.time()-start_time,
           'p_fmc': p_fmc,
           'B1':B[0:100000,:],
           'B2':B[100000:,:]}
with open(output_file, "wb") as f:
    pickle.dump(res_fMC, f)    

# Time=2031.9s


# Result analysis

In [6]:
with open(output_file, 'rb')as f:
    res_dic = pickle.load(f)
B = np.concatenate([res_dic['B1'], res_dic['B2']], axis=0)
p_fmc = res_dic['p_fmc']

In [7]:
snp_list = ['rs10501570', 'rs281357', 'rs2242330', 'rs1480597', 'rs6826751', 'rs4888984',
            'rs4862792', 'rs3775866', 'rs2235617', 'rs988421', 'rs7097094', 'rs999473',
            'rs1912373', 'rs1887279', 'rs2986574', 'rs11090762', 'rs6125829', 'rs7796855',
            'rs355477', 'rs3010040', 'rs2296713', 'rs355461', 'rs355506', 'rs355464',
            'rs1497430', 'rs11946612']
tau_fmc = md.bh(p_fmc[ind_snp], alpha=0.1)
h_fmc = (p_fmc[ind_snp] <= tau_fmc)
print(np.sum(h_fmc))
df_map_c4 = df_map.loc[ind_small]
for snp in snp_list:
    temp = df_map_c4['snp']==snp
    if np.sum(temp)>0:
        print('###')
        print(df_map_c4.loc[temp])
        print('decision', h_fmc[temp[ind_snp]])
        print('miss_prop=%0.4f, p_fmc='%(miss_prop[ind_small][temp]),
                                        p_fmc[temp])
        print('')

47
###
       chromosome        snp  start       end
100847          4  rs2242330      0  68276015
decision [ True]
miss_prop=0.0000, p_fmc= [7.999968e-06]

###
       chromosome        snp  start       end
100837          4  rs6826751      0  68262621
decision [ True]
miss_prop=0.0019, p_fmc= [1.5999936e-05]

###
       chromosome        snp  start        end
115749          4  rs4862792      0  188576499
decision [ True]
miss_prop=0.0483, p_fmc= [3.999984e-06]

###
       chromosome        snp  start       end
100842          4  rs3775866      0  68272946
decision [ True]
miss_prop=0.0000, p_fmc= [3.5999856e-05]

###
       chromosome       snp  start       end
100830          4  rs355477      0  68225291
decision [ True]
miss_prop=0.0074, p_fmc= [7.999968e-05]

###
       chromosome       snp  start       end
100828          4  rs355461      0  68209490
decision [ True]
miss_prop=0.0000, p_fmc= [7.999968e-05]

###
       chromosome       snp  start       end
100829          4  rs355

# Corresponding AMT result

In [8]:
temp_B = B[:,ind_snp]

In [9]:
start_time = time.time()
p_hat_ub, p_hat_lb, p_hat, tau_hat, n_amt = md.amt(md.f_sample_dummy, temp_B, n_hypothesis,
                                         alpha=0.1, n_fMC=n_fMC,
                                         verbose=True, delta=0.001)
h_amt = (p_hat_ub <= tau_hat)
print('# AMT: avg. MC samples = %0.1f, time=%0.2fs'%(np.mean(n_amt),
                                                     time.time()-start_time))
print('# D_AMT=%d, D_overlap=%d, D_fMC=%d'%(md.result_compare(h_amt, h_fmc)))
print('')

# Initialization parameters
# n_hypothesis=23915, n_fMC=250000, alpha=0.10, increment=1.10
# delta 0.001
# delta_CI 3.3642314968430376e-09
# r_hat=23915, tau_hat=0.1000
# batch_size [  100   111   122   134   147   162   178   195   215   236   260   286
   314   346   380   418   460   506   556   612   673   741   815   896
   985  1084  1192  1311  1443  1587  1745  1920  2112  2323  2555  2811
  3092  3401  3741  4115  4526  4979  5477  6025  6627  7290  8018  8820
  9702 10672 11740 12913 14205 15625 17188 18906 20797 22210]
# sum of batch size = 250000
# Initialization completed: time=-0.006s
# 0, avg_sample=100.0, tau=0.01632, r_hat=3903, n_u=3903, n_g=20012, n_l=0
# 1, avg_sample=118.1, tau=0.01042, r_hat=2491, n_u=2491, n_g=21424, n_l=0
# 2, avg_sample=130.8, tau=0.00777, r_hat=1859, n_u=1859, n_g=22056, n_l=0
# 3, avg_sample=141.2, tau=0.00619, r_hat=1480, n_u=1480, n_g=22435, n_l=0
# 4, avg_sample=150.3, tau=0.00536, r_hat=1283, n_u=1283, n_g=22632, n_l=0
# 5, avg_sample=159

# Directly run AMT 

In [10]:
temp_data = {'X':X[:,ind_small][:, ind_snp], 'y':y, 
             'Exp':Exp[:,ind_small][:, ind_snp],
             'r_Exp':r_Exp[:,ind_small][:, ind_snp],
             'chi2_obs':chi2_obs[ind_small][ind_snp]}

In [12]:
start_time = time.time()
p_hat_ub, p_hat_lb, p_hat, tau_hat, n_amt = md.amt(md.f_sample_chi2, temp_data, n_hypothesis,
                                         alpha=0.1, n_fMC=n_fMC,
                                         verbose=True, delta=0.001,
                                         random_state=0)
h_amt = (p_hat_ub <= tau_hat)
print('# AMT: avg. MC samples = %0.1f, time=%0.2fs'%(np.mean(n_amt),
                                                     time.time()-start_time))
print('# D_AMT=%d, D_overlap=%d, D_fMC=%d'%(md.result_compare(h_amt, h_fmc)))
print('')

# Initialization parameters
# n_hypothesis=23915, n_fMC=250000, alpha=0.10, increment=1.10
# delta 0.001
# delta_CI 3.3642314968430376e-09
# r_hat=23915, tau_hat=0.1000
# batch_size [  100   111   122   134   147   162   178   195   215   236   260   286
   314   346   380   418   460   506   556   612   673   741   815   896
   985  1084  1192  1311  1443  1587  1745  1920  2112  2323  2555  2811
  3092  3401  3741  4115  4526  4979  5477  6025  6627  7290  8018  8820
  9702 10672 11740 12913 14205 15625 17188 18906 20797 22210]
# sum of batch size = 250000
# Initialization completed: time=-0.004s
# 0, avg_sample=100.0, tau=0.01636, r_hat=3913, n_u=3913, n_g=20002, n_l=0
# 1, avg_sample=118.2, tau=0.01032, r_hat=2469, n_u=2469, n_g=21446, n_l=0
# 2, avg_sample=130.8, tau=0.00771, r_hat=1845, n_u=1845, n_g=22070, n_l=0
# 3, avg_sample=141.1, tau=0.00606, r_hat=1450, n_u=1450, n_g=22465, n_l=0
# 4, avg_sample=150.0, tau=0.00506, r_hat=1209, n_u=1209, n_g=22706, n_l=0
# 5, avg_sample=158